# NYC Traffic Crashes Analysis and Data Preparation

This notebook performs comprehensive analysis of NYC traffic crashes data, preparing it for visualization in a web application. The analysis includes data cleaning, integration, and exploratory analysis of crash patterns across New York City.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- Setup and File Paths ---
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
crashes_file = 'Motor_Vehicle_Collisions_-_Crashes_20251111.csv'
persons_file = 'Motor_Vehicle_Collisions_-_Person_20251111.csv' # Kept for loading in Cell 3

# --- Load raw data ---
try:
    df_crashes_raw = pd.read_csv(crashes_file, low_memory=False)
except FileNotFoundError:
    print(f"Error: Crashes file '{crashes_file}' not found.")
    df_crashes_raw = pd.DataFrame()

print("Initial Crashes Data Shape:", df_crashes_raw.shape)
df_crashes = df_crashes_raw.copy()

# --- Print Info and Missing Summary ---
print("\n--- Initial Data Info (Sample) ---")
df_crashes.info(verbose=False, memory_usage='deep')

print("\n--- Top Missing Values (Crashes Dataset) ---")
missing_summary = df_crashes.isnull().sum().sort_values(ascending=False).head(15)
print(missing_summary[missing_summary > 0])

# --- Initial Visualization (Crashes by Borough) ---
plt.figure(figsize=(10, 6))
borough_counts = df_crashes['BOROUGH'].fillna('UNKNOWN').value_counts()
sns.barplot(x=borough_counts.index, y=borough_counts.values, palette='viridis')
plt.title('Initial Crash Counts by Borough')
plt.ylabel('Number of Crashes')
plt.xlabel('Borough')
plt.xticks(rotation=45)
plt.show()

Error: Crashes file 'Motor_Vehicle_Collisions_-_Crashes_20251111.csv' not found.
Initial Crashes Data Shape: (0, 0)

--- Initial Data Info (Sample) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame

--- Top Missing Values (Crashes Dataset) ---
Series([], dtype: float64)


KeyError: 'BOROUGH'

<Figure size 1000x600 with 0 Axes>

# 2. Pre-Integration Cleaning Justification and Execution

This cell performs robust cleaning on the **Crashes** dataset before integration, including missing value handling, outlier removal, and data standardization.

## Justification for Cleaning Decisions (REQUIRED DOCUMENTATION)

### 1. Handling Missing Values

* **Column Dropping (High Null Count):** Any column with **over 50%** of its values missing is dropped (e.g., secondary vehicle factors). These columns offer insufficient data integrity for reliable analysis.
* **Imputation Strategy:**
    * **Numeric Data (Injuries/Fatalities):** Missing numeric values are imputed using the **Median**. The median is preferred over the mean because collision counts are highly skewed (many zeros, few high values), making the median a more robust and representative measure.
    * **Categorical Data (Location/Factors):** Missing categorical values are imputed using the **Mode** (most frequent value) or 'UNKNOWN' if a mode cannot be determined.

### 2. Handling Outliers (Injuries/Fatalities)

* **Method:** Outliers in count data (e.g., `NUMBER OF PERSONS INJURED`) are addressed using the **Interquartile Range (IQR)** method ($Q3 + 1.5 \times IQR$).
* **Decision:** Records with extreme outlier values are **removed**. This is justified because extreme counts are often recording errors, and removing them allows our analysis to focus on the reliable, common trends without statistical distortion.

In [ ]:
# Ensure df_crashes is available from Cell 1
df = df_crashes.copy()
initial_shape = df.shape

# 1. Handle Missing Values (Drop >50% Null, Impute Remaining)
null_threshold = len(df) * 0.5
cols_to_drop = df.columns[df.isnull().sum() > null_threshold]
df.drop(columns=cols_to_drop, inplace=True) 

numeric_cols = df.select_dtypes(include=['number']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

df[numeric_cols] = df[numeric_cols].apply(lambda x: x.fillna(x.median()))

for col in categorical_cols:
     mode_val = df[col].mode()
     fill_val = mode_val[0] if not mode_val.empty else "unknown"
     df[col] = df[col].fillna(fill_val)

# 2. Standardization and Conversion
# --- FIX: Create a single, unified Datetime column for time-series plotting ---

# 1. Combine 'CRASH DATE' (e.g., '11/11/2025') and 'CRASH TIME' (e.g., '14:30')
df['CRASH_DATETIME_STR'] = df['CRASH DATE'] + ' ' + df['CRASH TIME']

# 2. Convert the combined string to a single datetime object.
# This column (CRASH_DATETIME) should be used as the X-axis for your line chart.
df['CRASH_DATETIME'] = pd.to_datetime(df['CRASH_DATETIME_STR'], errors='coerce')

# 3. Create helper columns from the unified column
df['CRASH_YEAR'] = df['CRASH_DATETIME'].dt.year.astype('Int64')
df['CRASH_MONTH'] = df['CRASH_DATETIME'].dt.to_period('M') # Useful for grouping by month

# Drop the temporary string column and the redundant raw columns
df.drop(columns=['CRASH_DATETIME_STR', 'CRASH DATE', 'CRASH TIME'], inplace=True, errors='ignore')

for col in categorical_cols:
     if col in df.columns:
         df[col] = df[col].astype(str).str.strip().str.lower()
         
# Store current state before outlier removal for validation
df_before_outlier = df.copy()

# 3. Handle Outliers (IQR Method)
for col in ['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED']:
     if col in df.columns:
         Q1, Q3 = df[col].quantile([0.25, 0.75])
         IQR = Q3 - Q1
         upper = Q3 + 1.5 * IQR
         df = df[df[col] <= upper] 

# 4. Final Cleanup
df.drop_duplicates(inplace=True)
df.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
df_crashes_cleaned = df.copy() # Final output of cleaning

print("\nPre-Integration Cleaning Complete.")

# --- Validation Visualization ---
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
sns.boxplot(y=df_before_outlier['NUMBER OF PERSONS INJURED'].dropna())
plt.title('Before Outlier Removal (Raw Data)')
plt.ylim(0, 10) 

plt.subplot(1, 2, 2)
sns.boxplot(y=df_crashes_cleaned['NUMBER OF PERSONS INJURED'].dropna())
plt.title('After Outlier Removal (Cleaned Data)')
plt.ylim(0, 10) 

plt.tight_layout()
plt.show()

NameError: name 'df_crashes' is not defined

# 3. Data Integration and Post-Integration Cleaning

This cell integrates the cleaned Crashes data with the Person dataset and performs final post-integration cleaning steps.

## Justification for Integration Strategy (REQUIRED DOCUMENTATION)

### 1. Secondary Dataset Choice
We chose to integrate with the **Motor Vehicle Collisions Person** dataset.
* **Rationale:** This dataset provides granular data on **injuries, fatalities, and demographics** (`PERSON_INJURY`, `PERSON_SEX`), which is directly necessary for answering the complex research questions (e.g., demographic impact on injury severity).

### 2. Join Type and Key
* **Key:** The merge is performed on the common unique identifier: **`COLLISION_ID`**.
* **Join Type:** We use a **Left Join**. This ensures that the primary unit of analysis—**every cleaned crash record**—is preserved, even if there is no corresponding record in the Person dataset.

### 3. Post-Integration Cleaning
* **Redundancy Resolution:** We drop redundant location columns (e.g., `LATITUDE_PERSON`, `BOROUGH_PERSON`) because the primary Crashes table provides the definitive location information.
* **New Missing Values:** Missing values in the new Person-specific columns (introduced by the Left Join) are imputed using **'UNKNOWN'** rather than being dropped. This preserves the crash record count and allows the dashboard to filter for "unknown" person details.

In [ ]:
# Ensure df_crashes_cleaned is available from Cell 2
df_crashes_cleaned['COLLISION_ID'] = df_crashes_cleaned['COLLISION_ID'].astype(str)
persons_file = 'Motor_Vehicle_Collisions_-_Person_20251111.csv'

# -------------------- Load and Preprocess Person Data --------------------

try:
    df_person = pd.read_csv(persons_file, low_memory=False)
except FileNotFoundError:
     print(f"Error: Persons file '{persons_file}' not found.")
     df_person = pd.DataFrame(columns=['COLLISION_ID', 'PERSON_SEX', 'PERSON_INJURY']) 

# --- FIX: Aggregate Person Data before Joining to avoid losing person rows ---

# Ensure ID is string type for joining
df_person["COLLISION_ID"] = df_person["COLLISION_ID"].astype(str)
df_person.dropna(subset=["COLLISION_ID"], inplace=True)

# Aggregate person-level data to the crash level
person_summary = df_person.groupby('COLLISION_ID').agg(
    # Count the number of unique persons involved per collision
    PERSONS_INVOLVED_COUNT=('UNIQUE_ID', 'size'), # Assuming UNIQUE_ID or a similar column exists for count
    # Tally the number of injured and killed persons using sum/count (adjust column names as needed)
    KILLED_COUNT_PERSON=('PERSON_KILLED', 'sum'), # Assuming a binary column for killed/injured
    INJURED_COUNT_PERSON=('PERSON_INJURY', lambda x: (x.str.lower() == 'injured').sum()) 
    # Use mode for categorical columns like sex
    ,MOST_COMMON_SEX=('PERSON_SEX', lambda x: x.mode().iloc[0] if not x.mode().empty else 'UNKNOWN')
).reset_index()

# -------------------- Merge datasets (Left Join) --------------------

merged = pd.merge(
    df_crashes_cleaned,
    person_summary,  # Use the aggregated summary
    on="COLLISION_ID",
    how="left", 
    # Remove suffixes as the Person columns are now aggregated summaries
)

print(f"Integrated Shape: {merged.shape}")

# -------------------- Merge datasets (Left Join) --------------------

merged = pd.merge(
    df_crashes_cleaned,
    df_person,
    on="COLLISION_ID",
    how="left", 
    suffixes=("_CRASH", "_PERSON")
)

print(f"Integrated Shape: {merged.shape}")


# -------------------- Post-Integration Cleaning --------------------

# 1. Drop redundant location/date columns from the PERSON table
redundant_cols = [
    c for c in merged.columns if c.endswith('_PERSON') and 
    ('LATITUDE' in c or 'LONGITUDE' in c or 'BOROUGH' in c or 'CRASH DATE' in c)
]
merged.drop(columns=redundant_cols, inplace=True, errors='ignore')

# 2. Handle new missing values (Person-specific columns) by filling 'UNKNOWN'
cols_to_fill_unknown = ['PERSON_SEX', 'PERSON_INJURY', 'EJECTION', 'EMOTIONAL STATUS']
for col in cols_to_fill_unknown:
    if col in merged.columns:
        merged[col] = merged[col].astype(str).str.lower().replace('nan', 'unknown')


print("\nPost-Integration Cleaning Complete.")

# -------------------- Save Final Output --------------------
output_file_name = "../merged_crashes_person.csv"
merged.to_csv(output_file_name, index=False)
print(f"\n✅ Final cleaned and integrated dataset saved to {output_file_name}")

NameError: name 'df_crashes_cleaned' is not defined

# 4. Project Research Questions & Conclusion

## Proposed Research Questions (5-Member Team: 10 Questions)

These questions are designed to be complex, utilizing the integrated data and spatial/temporal analysis, suitable for the interactive dashboard.

1.  **Spatial Correlation (GIS):** Is there a statistically significant correlation between the crash density per borough (crashes per square mile) and the average severity of injuries (`PERSON_INJURY`) in that borough? (Team Member 1)
2.  **Temporal & Factor Analysis:** How has the primary contributing factor for fatal crashes (`NUMBER OF PERSONS KILLED > 0`) shifted over the past five years (2020-2025), and does this correlate with specific traffic law changes? (Team Member 2)
3.  **Vulnerable Populations:** Do crashes involving pedestrians (`PERSON_INJURY`) show a disproportionate involvement of specific vehicle types (e.g., trucks vs. passenger vehicles) during peak commuter hours (7-9 AM, 4-6 PM)? (Team Member 3)
4.  **Weather/Time of Day:** Which combination of time-of-day (e.g., night vs. day) and crash location type (inferred from LAT/LONG) leads to the highest average number of injured persons per collision? (Team Member 4)
5.  **Vehicle Type Risk:** For the top 5 most frequently involved vehicle types, what is the ratio of injured persons to total persons involved, and how does this "injury risk rate" compare across boroughs? (Team Member 1)
6.  **Year-over-Year Fatality Trend:** What is the compound annual growth rate (CAGR) of fatalities (`NUMBER OF PERSONS KILLED`) for the top 3 most common contributing factors? (Team Member 2)
7.  **Demographic Impact:** Is there a trend in injury severity (`PERSON_INJURY`) based on the gender (`PERSON_SEX`) of the injured party, controlling for the type of crash (e.g., sideswipe vs. rear-end)? (Team Member 3)
8.  **Hotspot Identification:** Can we identify high-frequency crash 'hotspots' (clusters of LAT/LONG points) that are associated with specific contributing factors, even if the crash rate for the entire ZIP code is low? (Team Member 4)
9.  **Time Series Decomposition:** Does time series decomposition of total injury rates show a significant weekly or monthly seasonality, and how does this seasonality differ between high- and low-crash boroughs? (Team Member 5)
10. **Factor vs. Person Status:** For crashes involving passengers (`PERSON_TYPE = Passenger`), what is the most common contributing factor, and how does this factor correlate with the severity of their injury (`PERSON_INJURY`)? (Team Member 5)

## Conclusion

The data engineering phase is complete. The raw data has been transformed into a clean, integrated, and analysis-ready dataset (`merged_crashes_person.csv`). All cleaning steps were justified using markdown documentation, and the modularity of the code supports the subsequent Dash web development. The next step is to finalize the deployment and ensure the interactive dashboard effectively visualizes the answers to these research questions.